In [1]:
import pandas as pd
import pickle
with open('train.pkl','rb') as f:
    train = pickle.load(f)
with open('test.pkl','rb') as f:
    test = pickle.load(f)
with open('df_train.pkl','rb') as f:
    df_train = pickle.load(f)
with open('df_test.pkl','rb') as f:
    df_test = pickle.load(f)

In [2]:
%%time
df_train.drop(['api','tid','index'],axis=1,inplace=True)
df_test.drop(['api','tid','index'],axis=1,inplace=True)

CPU times: total: 0 ns
Wall time: 12 ms


In [3]:
def get_apis(df):
    # 按照file_id进行分组
    group_fileid = df.groupby('file_id')    
    # 统计file_id 和对应的 api_sequence
    file_api = {}    
    # 计算每个file_id的api_sequence
    for file_id, file_group in group_fileid:
        # 针对file_id 按照线程tid 和 顺序index进行排序
        result = file_group.sort_values(['tid', 'index'], ascending=True)
        #得到api的调用序列
        api_sequence = ' '.join(result['api'])
        #print(api_sequence)
        file_api[file_id] = api_sequence
    return file_api

In [4]:
%%time
train_apis = get_apis(train)
test_apis=get_apis(test)

CPU times: total: 50.7 s
Wall time: 54.6 s


In [5]:
%%time
temp = pd.DataFrame.from_dict(train_apis, orient='index', columns=['api'])
temp = temp.reset_index().rename(columns={'index': 'file_id'})
df_train = df_train.merge(temp, on='file_id', how='left')
 
temp = pd.DataFrame.from_dict(test_apis, orient='index', columns=['api'])
temp = temp.reset_index().rename(columns={'index': 'file_id'})
df_test = df_test.merge(temp, on='file_id', how='left')
 
df_all = pd.concat([df_train, df_test], axis=0)
df_all.describe()

CPU times: total: 78.1 ms
Wall time: 103 ms


file_id         label      api_count   api_nunique       api_max  \
count  26842.000000  13887.000000   26842.000000  26842.000000  26842.000000   
mean    6719.090157      2.930655    6299.644885     49.188473    275.538857   
std     3888.360300      2.596575   17774.836337     30.210753     17.967609   
min        1.000000      0.000000       1.000000      1.000000     95.000000   
25%     3356.000000      0.000000     228.000000     24.000000    261.000000   
50%     6711.000000      3.000000    1812.000000     47.000000    281.000000   
75%    10066.000000      5.000000    7311.500000     71.000000    286.000000   
max    13887.000000      7.000000  888204.000000    175.000000    300.000000   

            api_min       api_std      api_mean    api_median       api_ptp  \
count  26842.000000  26807.000000  26842.000000  26842.000000  26842.000000   
mean      31.238581     49.062198    166.249683    165.519540    244.300276   
std       52.492967     15.670388     34.107152     44.295587     56.175952   
min        0.000000      0.000000     33.525295      6.000000      0.000000   
25%        6.000000     40.854431    147.546273    137.000000    241.000000   
50%        9.000000     49.604570    160.940639    152.000000    261.000000   
75%       25.000000     59.675767    181.213417    181.000000    275.000000   
max      281.000000    114.551299    299.639072    300.000000    300.000000   

       ...    tid_median       tid_ptp    index_count  index_nunique  \
count  ...  26842.000000  26842.000000   26842.000000   26842.000000   
mean   ...   2535.665524    686.551524    6299.644885    1779.856978   
std    ...    227.619883    854.599441   17774.836337    1934.252149   
min    ...    184.000000      0.000000       1.000000       1.000000   
25%    ...   2428.000000    100.000000     228.000000     125.000000   
50%    ...   2540.000000    384.000000    1812.000000     961.000000   
75%    ...   2676.000000    792.000000    7311.500000    3661.750000   
max    ...   5972.000000  20372.000000  888204.000000    5001.000000   

          index_max  index_min     index_std    index_mean  index_median  \
count  26842.000000    26842.0  26807.000000  26842.000000  26842.000000   
mean    1778.856978        0.0    510.268837    773.072243    720.564414   
std     1934.252149        0.0    555.853917    845.683594    807.020543   
min        0.000000        0.0      0.000000      0.000000      0.000000   
25%      124.000000        0.0     37.094474     61.407468     60.500000   
50%      960.000000        0.0    277.849780    459.500000    412.000000   
75%     3660.750000        0.0   1003.867770   1333.080881   1089.000000   
max     5000.000000        0.0   1654.531360   2500.000000   2500.000000   

          index_ptp  
count  26842.000000  
mean    1778.856978  
std     1934.252149  
min        0.000000  
25%      124.000000  
50%      960.000000  
75%     3660.750000  
max     5000.000000  

[8 rows x 26 columns]

In [6]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer(ngram_range=(1,3),min_df=0.01,max_df=0.99)
api_features = vec.fit_transform(df_all['api'])

CPU times: total: 2min 42s
Wall time: 2min 42s


In [7]:
df_apis=pd.DataFrame(api_features.toarray(),columns=vec.get_feature_names_out())

In [8]:
df_train_apis=df_apis[df_apis.index<=13886]
df_test_apis=df_apis[df_apis.index>13886]
df_test_apis.index=range(len(df_test_apis))
df_train_apis.head(1)

__exception__  __exception__ __exception__  \
0        0.00029                          0.0   

   __exception__ __exception__ __exception__  \
0                                        0.0   

   __exception__ __exception__ ldrloaddll  \
0                                     0.0   

   __exception__ __exception__ ntallocatevirtualmemory  \
0                                                0.0     

   __exception__ __exception__ ntclose  \
0                                  0.0   

   __exception__ __exception__ ntdelayexecution  __exception__ couninitialize  \
0                                           0.0                           0.0   

   __exception__ couninitialize ldrunloaddll  __exception__ getsysteminfo  \
0                                        0.0                          0.0   

   ...  wsastartup ntallocatevirtualmemory ntfreevirtualmemory  \
0  ...                                           0.000723        

   wsastartup regopenkeyexa  wsastartup regopenkeyexa regqueryvalueexa  \
0                       0.0                                        0.0   

   wsastartup regopenkeyexa regsetvalueexa  wsastartup socket  \
0                                      0.0           0.001996   

   wsastartup socket bind  wsastartup socket ntqueryattributesfile  \
0                0.003068                                      0.0   

   wsastartup wsastartup  wsastartup wsastartup findresourceexw  \
0               0.002783                                    0.0   

   wsastartup wsastartup socket  
0                      0.000796  

[1 rows x 7542 columns]

In [9]:
%%time
#合并特征
df_train.drop(['api'],axis=1,inplace=True)
df_test.drop(['api'],axis=1,inplace=True)
df_train =df_train.merge(df_train_apis,left_index=True,right_index=True)
df_test =df_test.merge(df_test_apis,left_index=True,right_index=True)

CPU times: total: 2.19 s
Wall time: 2.2 s


In [13]:
%%time
import lightgbm as lgb
clf = lgb.LGBMClassifier(
           num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='multiclass',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2024,
            n_estimators=2000, subsample=1, colsample_bytree=1)
clf.fit(df_train.drop(['label'],axis=1),df_train['label'])
result_lgb=clf.predict_proba(df_test)
result_lgb = pd.DataFrame(result_lgb,columns=['prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7'])
result_lgb['file_id']=df_test['file_id'].values

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.458509 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1177205
[LightGBM] [Info] Number of data points in the train set: 13887, number of used features: 7566
[LightGBM] [Info] Start training from score -1.025925
[LightGBM] [Info] Start training from score -3.320108
[LightGBM] [Info] Start training from score -2.451970
[LightGBM] [Info] Start training from score -2.829404
[LightGBM] [Info] Start training from score -4.933538
[LightGBM] [Info] Start training from score -1.174900
[LightGBM] [Info] Start training from score -3.294542
[LightGBM] [Info] Start training from score -2.234192
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

In [14]:
import psutil
mem = psutil.virtual_memory()
print('总内存',mem.total/1024/1024)
print('已使用内存',mem.used/1024/1024)
print('剩余内存',mem.free/1024/1024)
print('内存使用率',mem.percent)

总内存 16110.78515625
已使用内存 11730.49609375
剩余内存 4380.2890625
内存使用率 72.8


In [15]:
%%time
import xgboost as xgb
model_xgb = xgb.XGBClassifier(
    max_depth=9, learning_rate=0.005, n_estimators=2000, 
    objective='multi:softprob', tree_method="hist", device="cuda", 
    subsample=0.8, colsample_bytree=0.8, 
    min_child_weight=3, eval_metric='logloss', reg_lambda=0.5)
model_xgb.fit(df_train.drop(['label'],axis=1),df_train['label'])
result_xgb=model_xgb.predict_proba(df_test)
result_xgb = pd.DataFrame(result_xgb,columns=['prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7'])
result_xgb['file_id']=df_test['file_id'].values

XGBoostError: [08:56:42] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\tree\updater_gpu_hist.cu:781: Exception in gpu_hist: [08:56:42] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\data\../common/device_helpers.cuh:431: Memory allocation error on worker 0: bad allocation: cudaErrorMemoryAllocation: out of memory
- Free memory: 982515712
- Requested memory: 1442284480



In [12]:
%%time
result = result_lgb.copy()
weight_lgb = 0.6
weight_xgb = 0.4
result['prob0'] = result_lgb['prob0']*weight_lgb+result_xgb['prob0']*weight_xgb
result['prob1'] = result_lgb['prob1']*weight_lgb+result_xgb['prob1']*weight_xgb
result['prob2'] = result_lgb['prob2']*weight_lgb+result_xgb['prob2']*weight_xgb
result['prob3'] = result_lgb['prob3']*weight_lgb+result_xgb['prob3']*weight_xgb
result['prob4'] = result_lgb['prob4']*weight_lgb+result_xgb['prob4']*weight_xgb
result['prob5'] = result_lgb['prob5']*weight_lgb+result_xgb['prob5']*weight_xgb
result['prob6'] = result_lgb['prob6']*weight_lgb+result_xgb['prob6']*weight_xgb
result['prob7'] = result_lgb['prob7']*weight_lgb+result_xgb['prob7']*weight_xgb
columns=['file_id','prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7']
result.to_csv('./baseline2.csv',columns=columns,index=False)

NameError: name 'result_lgb' is not defined

In [16]:
%%time
result = result_lgb.copy()
result['prob0'] = result_lgb['prob0']
result['prob1'] = result_lgb['prob1']
result['prob2'] = result_lgb['prob2']
result['prob3'] = result_lgb['prob3']
result['prob4'] = result_lgb['prob4']
result['prob5'] = result_lgb['prob5']
result['prob6'] = result_lgb['prob6']
result['prob7'] = result_lgb['prob7']
columns=['file_id','prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7']
result.to_csv('./baseline2.csv',columns=columns,index=False)

CPU times: total: 156 ms
Wall time: 188 ms
